In [6]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import librosa
import librosa.display
import IPython.display
import numpy as np

MFCC_X = 20
MFCC_Y = 44

In [7]:
def _path(loc, directory, filename, redundant, format_):
    delimiter = '/'
    path = ( loc + directory + 
           filename + redundant + format_)
    return path

In [8]:
def _normalize(mfcc):
    return (mfcc - np.min(mfcc)) / (np.max(mfcc) - np.min(mfcc))

In [9]:
def _fit_size(arr, size):
    zeros = np.zeros( (len(arr), size - len(arr[0]) ) )
    arr = np.append(arr, zeros, axis = 1)
    arr.astype('float32')
    return arr

In [10]:
def _mfcc(path):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y = y, sr = sr)
    mfcc = _fit_size(mfcc, MFCC_Y)
    return  _normalize(mfcc)

In [11]:
class WordDataset(Dataset):
    def __init__(self, train = True):
        label_list = [0, 1]
        self.label_word = ['긍', '부']
        self.train = train
        num_data   =  10
        num_train  =  9
        num_test   =  1

        if self.train == True: 
            self.train_data   =  []
            self.train_label  =  []
            
            print("\n\n==== Train Data:")
            for item in label_list:
                print("===", end = "")
                for i in range(1, num_train + 1):
                    path = _path(loc = './Data/Audio', 
                              directory = self.label_word[item],filename = self.label_word[item],
                                 redundant =str(i), format_ = '.wav')
                    mfcc = _mfcc(path)
                    self.train_data.append(mfcc)
                    self.train_label.append(item)
                    
            self.train_label = np.array(self.train_label)
            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape(num_train * 4, MFCC_X, MFCC_Y) # 3600개에 (20,44) mfcc
            print("=== Dataset Download Complete !!")
            
        else:
            self.test_data   =  []
            self.test_label  =  []
            
            print("\n\n=== Test Data:")
            for item in label_list:
                print("===", end = "")
                for i in range(num_train + 1, num_data + 1):              
                    path = _path(loc = '.\\dataset', 
                              directory = self.label_word[item],filename = self.label_word[item],
                                 redundant = str(i), format_ = '.wav')
                    mfcc = _mfcc(path)
                    self.test_data.append(mfcc)
                    self.test_label.append(item)
                    
            self.test_label = np.array(self.test_label)
            self.test_data = np.concatenate(self.test_data)
            self.test_data = self.test_data.reshape(num_test * 4, MFCC_X, MFCC_Y) # 3600개에 (20,44) mfcc
            print("=== Dataset Download Complete !!")
        
    def __getitem__(self, index):
        if self.train:
            return self.train_data[index], self.train_label[index]
        else:
            return self.test_data[index], self.test_label[index]
    
    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

In [12]:
train_dataset = WordDataset(train = True)
test_dataset = WordDataset(train = False)

train_loader = DataLoader(train_dataset, batch_size = 10, shuffle = True, num_workers = 0, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = 10, shuffle = False, num_workers = 0, drop_last = True)



==== Train Data:
===

C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: './Data/Audio긍긍1.wav'